# Projeto PontUau pt.3 – Adição de Distância Geográfica no Dataset


## Origem dos dados

Notebook anterior à esse: **LINK PARA A LOCALIZACAO DE TAL ARQUIVO NO REPOSITÓRIO DO GITHUB**

## Objetivo do Notebook

Adicionar uma variável `distancia_km` no dataset

## Descrição do Projeto

Na análise de malha aérea, previsão de demanda, custos operacionais ou modelagem de atrasos,
a **distância entre origem e destino** é uma variável fundamental.

Bases públicas da ANAC não trazem esse dado de forma explícita.
Portanto, este projeto resolve um problema clássico:
> transformar dados brutos operacionais em informação analítica de alto valor.

## Processo

### Instalando e importando as bibliotecas necessárias para trazer os dados de cidade e KM

In [ ]:
# Instalando

!pip install airportsdata
!pip install geopy

In [ ]:
# Importando

import pandas as pd
import gdown
from geopy.distance import great_circle
import numpy as np
from geopy.distance import great_circle
import numpy as np
import airportsdata

### Criando um dataset a parte para facilitar o processo

In [ ]:
# Trazendo a base mundial de aeroportos para criamos um df de coordenadas

airports = airportsdata.load('ICAO')

In [ ]:
# Filtrando apenas aeroportos brasileiros

lista_aeroportos = []

for code, data in airports.items():
    if data['country'] == 'BR':
        lista_aeroportos.append({
            'icao_code': code,
            'iata_code': data['iata'],
            'nome': data['name'],
            'cidade': data['city'],
            'estado': data['subd'],
            'latitude': data['lat'],
            'longitude': data['lon']
        })


df_coords = pd.DataFrame(lista_aeroportos)


print(f"Total de aeroportos brasileiros mapeados: {len(df_coords)}")
df_coords.head()



Total de aeroportos brasileiros mapeados: 2824


,icao_code,iata_code,nome,cidade,estado,latitude,longitude
0,SBAA,CDJ,Conceicao do Araguaia Airport,Conceicao Do Araguaia,Pará,-8.34835,-49.30150
1,SBAE,JTC,Bauru-Arealva Airport,Bauru,São Paulo,-22.15783,-49.06833
2,SBAF,,Campo Delio Jardim de Mattos Airport,Rio De Janeiro,Rio de Janeiro,-22.87510,-43.38470
3,SBAM,,Amapá Airport,Amapá,Amapá,2.07751,-50.85820
4,SBAN,,Base Aerea Airport,Anapolis,Goiás,-16.22920,-48.96430


### Trazendo o dataset criado anteriormente para este Notebook

In [ ]:
vra_pre_ml = pd.read_parquet('/content/dataset_proprio_pt1.parquet')

Downloading...
From: https://drive.google.com/uc?id=1pLeneqhtzaXpDXEqCt19kxA3hzPwopY-
To: /content/vra_pre_ml.parquet
100%|██████████| 45.5M/45.5M [00:00<00:00, 151MB/s]


### Fazendo um merge dos datasets e preparando os dados para trazer as coordenadas de nome dos aeroportos e cidades

In [ ]:
coord_origem = df_coords[['icao_code', 'nome', 'cidade', 'estado', 'latitude', 'longitude']].copy()
coord_origem.columns = ['icao_code_origem', 'nome_origem', 'cidade_origem', 'estado_origem', 'latitude_origem', 'longitude_origem']


vra_pre_ml_cidades = pd.merge(
    vra_pre_ml,
    coord_origem,
    left_on='icao_aerodromo_origem',
    right_on='icao_code_origem',
    how='left'
)


coord_destino = df_coords[['icao_code', 'nome', 'cidade', 'estado', 'latitude', 'longitude']].copy()
coord_destino.columns = ['icao_code_destino', 'nome_destino', 'cidade_destino', 'estado_destino', 'latitude_destino', 'longitude_destino']


vra_pre_ml_cidades = pd.merge(
    vra_pre_ml_cidades,
    coord_destino,
    left_on='icao_aerodromo_destino',
    right_on='icao_code_destino',
    how='left'
)

### Calculando a distância entre aeroportos

In [ ]:
# Definindo uma função para calcular a distância de maneira segura, lidando com potenciais valores NaN

def calculate_distance(row):
    try:
        if pd.isna(row['latitude_origem']) or pd.isna(row['longitude_origem']) or \
           pd.isna(row['latitude_destino']) or pd.isna(row['longitude_destino']):
            return np.nan

        origin_coords = (row['latitude_origem'], row['longitude_origem'])
        destination_coords = (row['latitude_destino'], row['longitude_destino'])
        return great_circle(origin_coords, destination_coords).km
    except Exception:
        return np.nan

# Aplicando a função criada
vra_pre_ml_cidades['distancia_km'] = vra_pre_ml_cidades.apply(calculate_distance, axis=1)

# Exibindo o DataFrame 'vra_pre_ml_cidades' com a coluna 'distancia_km'
display(vra_pre_ml_cidades[['icao_aerodromo_origem', 'icao_aerodromo_destino', 'nome_origem', 'cidade_origem', 'nome_destino', 'cidade_destino', 'distancia_km']].head())

,icao_aerodromo_origem,icao_aerodromo_destino,nome_origem,cidade_origem,nome_destino,cidade_destino,distancia_km
0,SBGR,SBCY,Guarulhos - Governador Andre Franco Montoro In...,Sao Paulo,Marechal Rondon Airport,Cuiaba,1329.535137
1,SBCT,SBFI,Afonso Pena Airport,Curitiba,Cataratas International Airport,Foz Do Iguacu,532.585773
2,SBCT,SBFI,Afonso Pena Airport,Curitiba,Cataratas International Airport,Foz Do Iguacu,532.585773
3,SBCT,SBFI,Afonso Pena Airport,Curitiba,Cataratas International Airport,Foz Do Iguacu,532.585773
4,SBCT,SBFI,Afonso Pena Airport,Curitiba,Cataratas International Airport,Foz Do Iguacu,532.585773


In [ ]:
# Verificando as informações

vra_pre_ml_cidades.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1498559 entries, 0 to 1498558
Data columns (total 42 columns):
 #   Column                   Non-Null Count    Dtype         
---  ------                   --------------    -----         
 0   icao_empresa_aerea       1498559 non-null  object        
 1   numero_voo               1498559 non-null  object        
 2   codigo_autorizacao_di    1498559 non-null  object        
 3   codigo_tipo_linha        1498559 non-null  object        
 4   icao_aerodromo_origem    1498559 non-null  object        
 5   icao_aerodromo_destino   1498559 non-null  object        
 6   partida_prevista         1498559 non-null  datetime64[ns]
 7   partida_real             1498559 non-null  datetime64[ns]
 8   chegada_prevista         1498559 non-null  datetime64[ns]
 9   chegada_real             1498559 non-null  datetime64[ns]
 10  situacao_voo             1498559 non-null  object        
 11  tempo_voo_estimado       1498559 non-null  float64       
 12  

## Resultado Final

Ao final do processo, temos o Dataset enriquecido com **distância em km**

Como próximo passo, o projeto avança para a etapa de modelagem preditiva, que será conduzida em um outro Notebook.

Nessa fase, utiliza-se este dataset consolidado para aplicação de técnicas de pré-processamento, treinamento de modelos de classificação e avaliação de desempenho, com foco na previsão da probabilidade de atraso antes da realização do voo.

In [ ]:
# Exportanto o Dataset Próprio Final

vra_pre_ml_cidades.to_parquet(
    "dataset_proprio.parquet",
    index=False
)